In [1]:
# Importamos re para expresiones regulares
import os
import settings
from preprocessor import *

In [2]:
settings.initialize()

In [3]:
#!pip install -r {settings.PUNCTUATOR2TFT_DIR}/requirements.txt

In [4]:
TEST_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.test.en')
CHECK_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.check.en')
TRAIN_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.train.en')

TEST_PREP_PATH  = prepare_file(TEST_RAW_PATH,'test.prepared.txt')
CHECK_PREP_PATH = prepare_file(CHECK_RAW_PATH,'check.prepared.txt',allow_duplicates=False)
TRAIN_PREP_PATH = prepare_file(TRAIN_RAW_PATH,'train.prepared.txt',lowercase=True)

In [5]:
from preprocessor import train_dev_test_split

In [6]:
TRAIN_TRAIN_PREP_PATH, TRAIN_DEV_PREP_PATH, TRAIN_TEST_PREP_PATH = train_dev_test_split(TRAIN_PREP_PATH, train_split = 0.7, dev_split = 0.15)

In [8]:
!python {settings.PUNCTUATOR2TF2_DIR}/data.py {settings.DATA_PREPARED_DIR}

Vocabulary size: 44964
Vocabulary "data/preprocessed/vocabulary" size: 44964
Vocabulary "data/preprocessed/punctuations" size: 7
0.22% UNK-s in data/preprocessed/train
Vocabulary "data/preprocessed/vocabulary" size: 44964
Vocabulary "data/preprocessed/punctuations" size: 7
0.89% UNK-s in data/preprocessed/dev
Vocabulary "data/preprocessed/vocabulary" size: 44964
Vocabulary "data/preprocessed/punctuations" size: 7
0.87% UNK-s in data/preprocessed/test


In [12]:
model_name = 'punctuator'
n_layers = 50
lr = 0.02
model_full_name = 'Model_{}_h{}_lr{}.pcl'.format(model_name,n_layers,lr)

In [13]:
!python {settings.PUNCTUATOR2TF2_DIR}/main.py {model_name} {n_layers} {lr}

50 0.02 Model_punctuator_h50_lr0.02.pcl
Building model ...
Vocabulary "data/preprocessed/vocabulary" size: 44964
2022-06-19 15:00:32.720381: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Vocabulary "data/preprocessed/vocabulary" size: 44964
Vocabulary "data/preprocessed/punctuations" size: 7
[TensorShape([44964, 50]), TensorShape([50, 7]), TensorShape([1, 7]), TensorShape([50, 100]), TensorShape([100, 100]), TensorShape([1, 100]), TensorShape([100]), TensorShape([50, 50]), TensorShape([100, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([100, 100]), TensorShape([50, 100]), TensorShape([1, 100]), TensorShape([100, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([50, 100]), TensorShape([50, 100]), TensorShape([1, 100]), TensorShape([50, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([50, 100]), Tenso

In [7]:
output_predicted_file_path = os.path.join(settings.PREDICTED_DIR,'test.punctuated2tf2.raw.txt')

In [15]:
!python {settings.PUNCTUATOR2TF2_DIR}/punctuator.py ./{model_full_name} {TEST_RAW_PATH} {output_predicted_file_path}

Vocabulary "data/preprocessed/vocabulary" size: 44964
Loading model parameters...
2022-06-19 15:39:39.244991: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Vocabulary "data/preprocessed/vocabulary" size: 44964
Vocabulary "data/preprocessed/punctuations" size: 7
[TensorShape([44964, 50]), TensorShape([50, 7]), TensorShape([1, 7]), TensorShape([50, 100]), TensorShape([100, 100]), TensorShape([1, 100]), TensorShape([100]), TensorShape([50, 50]), TensorShape([100, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([100, 100]), TensorShape([50, 100]), TensorShape([1, 100]), TensorShape([100, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([50, 100]), TensorShape([50, 100]), TensorShape([1, 100]), TensorShape([50, 50]), TensorShape([50, 50]), TensorShape([1, 50]), TensorShape([50, 100]), TensorShape([50, 100]), TensorShape(

In [8]:
TEST_PREDICT_PREP_PATH  = prepare_file(output_predicted_file_path,'test.punctuated2tf2.prepared.txt',lowercase=False,allow_duplicates=False,token_punct=False)

In [34]:
TEST_PREDICT_PREP_PATH

'./data/prepared/test.punctuated2tf2.prepared.txt'

In [9]:
!python ./punctuator2tf2/error_calculator.py /content/data/prepared/check.prepared.txt ./data/prepared/test.punctuated2tf2.prepared.txt

Vocabulary "data/preprocessed/punctuations" size: 7
----------------------------------------------
PUNCTUATION      PRECISION RECALL    F-SCORE  
?QUESTIONMARK    42.400    13.200    20.200   
,COMMA           65.100    60.800    62.900   
;SEMICOLON       11.100    0.800     1.400    
!EXCLAMATIONMARK 0.000     0.000     nan      
.PERIOD          50.800    21.600    30.300   
:COLON           50.800    10.200    17.000   
----------------------------------------------
Overall          60.500    40.000    48.200   
Err: 9.95%
SER: 72.2%


In [10]:
from evaluator import evaluate_punctuated

In [11]:
evaluate_punctuated(CHECK_PREP_PATH,TEST_RAW_PATH,TEST_PREDICT_PREP_PATH)

MÉTRICAS
precision global:  0.8838893706975988
recall global:  0.8086521534303638
F1 global:  0.844598531639669
precision media:  0.893604385062655
recall medio:  0.7076767808421794
F1 medio:  0.7898463905434807
rendimiento:  0.05235711305798915
número de instancias en el corpus:  14382


{'F1_global': 0.844598531639669,
 'F1_mean': 0.7898463905434807,
 'precision_global': 0.8838893706975988,
 'precision_mean': 0.893604385062655,
 'recall_global': 0.8086521534303638,
 'recall_mean': 0.7076767808421794,
 'score': 0.05235711305798915}